In [ ]:
from os import path, makedirs, rename, remove
from time import time
import argparse
import pickle

import numpy as np
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer

In [ ]:
train_data = pd.read_csv('./data_fake_news/train.csv')
test_data = pd.read_csv('./data_fake_news/test.csv')

In [ ]:
test_data.head()

In [ ]:
descriptions = train_data['text']
train_size = int(len(descriptions) * .8)
labels = train_data['label']
train_description = descriptions[:train_size].astype('str')
train_labels = labels[:train_size]
test_description = descriptions[train_size:].astype('str')
test_labels = labels[train_size:]

In [ ]:
top_labels = ['0', '1']
description_embeddings = hub.text_embedding_column(
  "text", 
  module_spec="https://tfhub.dev/google/universal-sentence-encoder/2"
)

In [ ]:
encoder = LabelBinarizer()
encoder.fit_transform(train_labels)
train_encoded = encoder.transform(train_labels)
test_encoded = encoder.transform(test_labels)
num_classes = len(encoder.classes_)

In [ ]:
binary_label_head = tf.contrib.estimator.binary_classification_head(
                        loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
                    )

In [ ]:
estimator = tf.contrib.estimator.DNNEstimator(
    head=binary_label_head,
    hidden_units=[64,10],
    feature_columns=[description_embeddings]
)

In [85]:
estimator_linear_combined = tf.contrib.estimator.DNNLinearCombinedEstimator(
    head=binary_label_head,
    dnn_hidden_units=[64,10],
    dnn_feature_columns=[description_embeddings]
)

I0328 14:28:40.211071 24732 tf_logging.py:116] Using default config.
W0328 14:28:40.215081 24732 tf_logging.py:126] Using temporary folder as model directory: C:\Users\I349732\AppData\Local\Temp\tmpn8qc1fes
I0328 14:28:40.217086 24732 tf_logging.py:116] Using config: {'_num_worker_replicas': 1, '_session_config': None, '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_master': '', '_task_type': 'worker', '_save_summary_steps': 100, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_is_chief': True, '_task_id': 0, '_global_id_in_cluster': 0, '_service': None, '_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001C622F4E9E8>, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': None, '_model_dir': 'C:\\Users\\I349732\\AppData\\Local\\Temp\\tmpn8qc1fes'}


In [86]:
features = {
  "text": np.array(train_description).astype(np.str)
}
labels = np.array(train_encoded).astype(np.int32)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    features, 
    labels, 
    shuffle=True, 
    batch_size=32, 
    num_epochs=20
)

In [87]:
tf.logging.set_verbosity(tf.logging.INFO)
estimator_linear_combined.train(input_fn=train_input_fn)

I0328 14:29:02.829283 24732 tf_logging.py:116] Calling model_fn.
I0328 14:29:11.726186 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_0:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47\\variables\\variables' with Embeddings_en/sharded_0
I0328 14:29:11.746273 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_1:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47\\variables\\variables' with Embeddings_en/sharded_1
I0328 14:29:12.069132 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_10:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a3847

I0328 14:29:12.281748 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_1/bias:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47\\variables\\variables' with SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_1/bias
I0328 14:29:12.292779 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_1/weights:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47\\variables\\variables' with SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_1/weights
I0328 14:29:12.301803 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_

I0328 14:45:12.899289 24732 tf_logging.py:116] global_step/sec: 3.15157
I0328 14:45:12.905298 24732 tf_logging.py:116] loss = 0.671467, step = 2701 (31.716 sec)
I0328 14:45:42.084892 24732 tf_logging.py:116] global_step/sec: 3.42635
I0328 14:45:42.089908 24732 tf_logging.py:116] loss = 0.677121, step = 2801 (29.186 sec)
I0328 14:46:10.917565 24732 tf_logging.py:116] global_step/sec: 3.46829
I0328 14:46:10.920572 24732 tf_logging.py:116] loss = 0.685724, step = 2901 (28.831 sec)
I0328 14:46:39.850501 24732 tf_logging.py:116] global_step/sec: 3.45627
I0328 14:46:39.854513 24732 tf_logging.py:116] loss = 0.67564, step = 3001 (28.934 sec)
I0328 14:47:12.458210 24732 tf_logging.py:116] global_step/sec: 3.06685
I0328 14:47:12.470243 24732 tf_logging.py:116] loss = 0.681831, step = 3101 (32.616 sec)
I0328 14:47:41.688939 24732 tf_logging.py:116] global_step/sec: 3.42094
I0328 14:47:41.692951 24732 tf_logging.py:116] loss = 0.678009, step = 3201 (29.223 sec)
I0328 14:48:11.783968 24732 tf_logg

I0328 15:09:37.355380 24732 tf_logging.py:116] loss = 0.62531, step = 7601 (29.188 sec)
I0328 15:10:10.767970 24732 tf_logging.py:116] global_step/sec: 2.99252
I0328 15:10:10.790021 24732 tf_logging.py:116] loss = 0.57371, step = 7701 (33.435 sec)
I0328 15:10:40.295479 24732 tf_logging.py:116] global_step/sec: 3.38667
I0328 15:10:40.300492 24732 tf_logging.py:116] loss = 0.627666, step = 7801 (29.510 sec)
I0328 15:10:45.026058 24732 tf_logging.py:116] Saving checkpoints for 7819 into C:\Users\I349732\AppData\Local\Temp\tmpn8qc1fes\model.ckpt.
I0328 15:11:24.857712 24732 tf_logging.py:116] global_step/sec: 2.2441
I0328 15:11:24.861728 24732 tf_logging.py:116] loss = 0.655907, step = 7901 (44.561 sec)
I0328 15:11:53.637577 24732 tf_logging.py:116] global_step/sec: 3.47465
I0328 15:11:53.641588 24732 tf_logging.py:116] loss = 0.619349, step = 8001 (28.780 sec)
I0328 15:12:22.226966 24732 tf_logging.py:116] global_step/sec: 3.49768
I0328 15:12:22.239929 24732 tf_logging.py:116] loss = 0.62

In [88]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"text": np.array(test_description).astype(np.str)}, test_encoded.astype(np.int32), shuffle=False)

estimator.evaluate(input_fn=eval_input_fn)

I0328 16:59:45.888293 24732 tf_logging.py:116] Calling model_fn.
I0328 16:59:58.972562 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_0:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47\\variables\\variables' with Embeddings_en/sharded_0
I0328 16:59:58.988189 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_1:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47\\variables\\variables' with Embeddings_en/sharded_1
I0328 16:59:58.988189 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_10:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a3847

I0328 16:59:59.193506 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_1/bias:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47\\variables\\variables' with SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_1/bias
I0328 16:59:59.202530 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_1/weights:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47\\variables\\variables' with SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_1/weights
I0328 16:59:59.211554 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_

{'accuracy': 0.88149041,
 'accuracy_baseline': 0.50793266,
 'auc': 0.95476878,
 'auc_precision_recall': 0.95874542,
 'average_loss': 0.27444336,
 'global_step': 3609,
 'label/mean': 0.50793266,
 'loss': 0.27423015,
 'prediction/mean': 0.48924395}

In [ ]:
raw_test = [
    "The BBC produced spoof on the â€œReal Housewivesâ€ TV programmes, which has a comedic Islamic State twist, has been criticised by Leftists and Muslims who claim the sketch is offensive. [The BBC released the trailer earlier this week and were immediately slammed by those on the left and Muslims who thought that making fun of the brides of members of the terror group was out of the bounds of conventional humour. The sketch is part of a new programme called â€œRevoltingâ€ written by Jolyon Rubinstein and Heydon Prowse which, according to the BBC, is â€œsatirising the state of the nation. â€  LOLOLOL pic. twitter.   â€”   Raheem Kassam (@RaheemKassam) January 4, 2017,  The BBC2 Facebook page was inundated with criticism. One user wrote, â€œIâ€™m mortified that the BBC had produced such a programme. This is simply bad taste. The fact it is a comedy makes it even more worrying that humour should be associated with the actions of ISIS. Is this really what TV licenses are funding! ?â€ Others took a different point of view including a Muslim saying, â€œAs a Muslim I find this HILARIOUS! Brilliant! The satire is on point highlighting the pathetic ideals of a pathetic group like ISIS. â€ Not all Muslims agreed with the sentiment and users on Twitter expressed just as much outrage for the sketch. One man, a Bangladeshi, said â€œThe BBC really made a satirical show called â€˜The Real Housewives of ISISâ€™ while the real housewives of ISIS are being raped and abused daily. â€   The BBC really made a satirical show called â€The Real Housewives of ISISâ€ while the real housewives of ISIS are being raped and abused daily,  â€”   Meraj. (@UncleMeraj) January 4, 2017,  Another wrote, â€œâ€˜Real Housewives of ISISâ€™ will make Hijabis feel more isolated n targeted by Islamophobes. Thanks @BBC for adding to the negative stereotype. â€ â€™Real Housewives of ISISâ€™ will make Hijabis feel more isolated n targeted by Islamophobes. Thanks @BBC for adding to the negative stereotype,  â€”   aÑ•Ð½ (@AshKaneSkittles) January 4, 2017,  Leftists also articulated how offended they were that the public broadcaster would dare create such a sketch mocking Islamic State. Some questioned whether the use of taxpayer money via the TV license fee should go toward the funding of the programme. As if peopleâ€™s TV license in this country is going towards funding the production of a programme called â€the real housewives of ISISâ€. Wow,  â€”   Cameron Edgar (@CammyyyEdgar) January 4, 2017,    The video itself has already been viewed millions of times on Facebook and other social media platforms.  The clip shows several women in hijabs talking to each other in a    house taking selfies and showing off their suicide belts to each other. One woman even mentions that she hadnâ€™t come from Birmingham â€œto do thisâ€ as she scrubbed the floor of the home. Women joining Islamic State and travelling to Syria to become brides has become a real problem in European countries as the terror group promises young girls a more glamorous lifestyle. In 2014, eight schoolgirls from Bethnal Green travelled to Syria to become brides of Islamic State fighters, all of them under the age of 18."
]

In [ ]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn({"text": np.array(raw_test).astype(np.str)}, shuffle=False)
results = estimator.predict(predict_input_fn)

In [ ]:
print(list(results)[0])